## go to Azure AI Foundry -> Management center -> copy API key and Endpoint

#### Resource: [https://learn.microsoft.com/en-us/training/modules/fine-tune-azure-databricks/1-introduction](url)

In [0]:
 import os

 os.environ["AZURE_OPENAI_API_KEY"] = "4aQ5GYmjbe5e6crb1zPrsrRyXH3nlBn2xxGYZkdrvhZJ1YSUrgH9JQQJ99BHACHrzpqXJ3w3AAAAACOG2uwx"
 os.environ["AZURE_OPENAI_ENDPOINT"] = "https://aifoundrysachin.cognitiveservices.azure.com/"
 os.environ["AZURE_OPENAI_API_VERSION"] = "2024-05-01-preview"

#### Both training_set.jsonl and validation_set.jsonl are made of different conversation examples between user and assistant that will serve as data points for training and validating the fine-tuned model. While the datasets for this exercise are considered small, it is important to keep in mind when working with bigger datasets that the LLMs have a maximum context length in terms of tokens. Therefore, you can verify the token count of your datasets before training your model and revise them if necessary.



In [0]:
import json
import tiktoken
import numpy as np
from collections import defaultdict

encoding = tiktoken.get_encoding("cl100k_base")

def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n##### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")

files = ['/dbfs/FileStore/fine_tuning/training_set.jsonl', '/dbfs/FileStore/fine_tuning/training_set.jsonl']

for file in files:
    print(f"File: {file}")
    with open(file, 'r', encoding='utf-8') as f:
        dataset = [json.loads(line) for line in f]

    total_tokens = []
    assistant_tokens = []

    for ex in dataset:
        messages = ex.get("messages", {})
        total_tokens.append(num_tokens_from_messages(messages))
        assistant_tokens.append(num_assistant_tokens_from_messages(messages))

    print_distribution(total_tokens, "total tokens")
    print_distribution(assistant_tokens, "assistant tokens")
    print('*' * 75)

File: /dbfs/FileStore/fine_tuning/training_set.jsonl

##### Distribution of total tokens:
min / max: 47, 62
mean / median: 52.1, 50.5

##### Distribution of assistant tokens:
min / max: 13, 30
mean / median: 17.6, 15.5
***************************************************************************
File: /dbfs/FileStore/fine_tuning/training_set.jsonl

##### Distribution of total tokens:
min / max: 47, 62
mean / median: 52.1, 50.5

##### Distribution of assistant tokens:
min / max: 13, 30
mean / median: 17.6, 15.5
***************************************************************************


##### As a reference, the model used in this exercise, GPT-4o, has the context limit (total number of tokens in the input prompt and the generated response combined) of 128K tokens.

##### Upload fine-tuning files to Azure OpenAI: Before you start to fine-tune the model, you need to initialize an OpenAI client and add the fine-tuning files to its environment, generating file IDs that will be used to initialize the job.

In [0]:
 import os
 from openai import AzureOpenAI

 client = AzureOpenAI(
   azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
   api_key = os.getenv("AZURE_OPENAI_API_KEY"),
   api_version = "2024-05-01-preview"  # This API version or later is required to access seed/events/checkpoint features
 )

 training_file_name = '/dbfs/FileStore/fine_tuning/training_set.jsonl'
 validation_file_name = '/dbfs/FileStore/fine_tuning/training_set.jsonl'

 training_response = client.files.create(
     file = open(training_file_name, "rb"), purpose="fine-tune"
 )
 training_file_id = training_response.id

 validation_response = client.files.create(
     file = open(validation_file_name, "rb"), purpose="fine-tune"
 )
 validation_file_id = validation_response.id

 print("Training file ID:", training_file_id)
 print("Validation file ID:", validation_file_id)

Training file ID: file-7357a0e2c2c845e6baa7aa84811b4bd8
Validation file ID: file-c4c6a607668c41e1ab704b734066b708


In [0]:
response = client.fine_tuning.jobs.create(
    training_file = training_file_id,
    validation_file = validation_file_id,
    model = "gpt-4o",
    seed = 105 # seed parameter controls reproducibility of the fine-tuning job. If no seed is specified one will be generated automatically.
)

job_id = response.id

### Fine-tuning a model can take over 60 minutes

##### Submit fine-tuning job: Now that the fine-tuning files have been successfully uploaded you can submit your fine-tuning training job. It isn’t unusual for training to take more than an hour to complete. Once training is completed, you can see the results in Azure AI Foundry by selecting the Fine-tuning option in the left pane.

##### NOTE: You can also monitor the job status in AI Foundry by selecting Fine-tuning in the left sidebar
##### jump to Azure AI Foundry -> Fine Tuniing -> Check your model in Queue "Fine-tune with your own data" 

#### Wait for the job status changes to **succeeded**

In [0]:
print("Job ID:", response.id)
print("Status:", response.status)

Job ID: ftjob-4ab71dc2365443ea8b012b1d1db8e8d3
Status: running


#### Once the job status changes to succeeded, run the following code to get the final results:

In [0]:
response = client.fine_tuning.jobs.retrieve(job_id)

print(response.model_dump_json(indent=2))
fine_tuned_model = response.fine_tuned_model

{
  "id": "ftjob-4ab71dc2365443ea8b012b1d1db8e8d3",
  "created_at": 1755584359,
  "error": null,
  "fine_tuned_model": null,
  "finished_at": null,
  "hyperparameters": {
    "batch_size": 1,
    "learning_rate_multiplier": 1.0,
    "n_epochs": 10
  },
  "model": "gpt-4o-2024-08-06",
  "object": "fine_tuning.job",
  "organization_id": null,
  "result_files": [
    "file-d51fecbeff754eb09e2a127ed6923e74"
  ],
  "seed": 105,
  "status": "running",
  "trained_tokens": null,
  "training_file": "file-7357a0e2c2c845e6baa7aa84811b4bd8",
  "validation_file": "file-c4c6a607668c41e1ab704b734066b708",
  "estimated_finish": null,
  "integrations": null,
  "metadata": null,
  "method": null
}


##### Review the json response and note the unique name generated in the "fine_tuned_model" field. It will be used in the following optional task.

##### Deploy fine-tuned model: Now that you have a fine-tuned model, you can deploy it as a customized model and use it like any other deployed model in either the Chat Playground of Azure AI Foundry, or via the chat completion API.

##### replacing the placeholders <YOUR_SUBSCRIPTION_ID>, <YOUR_RESOURCE_GROUP_NAME>, <YOUR_AZURE_OPENAI_RESOURCE_NAME>, and <FINE_TUNED_MODEL>:

In [0]:
import json
import requests

# token = os.getenv("TEMP_AUTH_TOKEN")
# subscription = "<YOUR_SUBSCRIPTION_ID>"
# resource_group = "<YOUR_RESOURCE_GROUP_NAME>"
# resource_name = "<YOUR_AZURE_OPENAI_RESOURCE_NAME>"
# model_deployment_name = "gpt-4o-ft"

token = os.getenv("TEMP_AUTH_TOKEN")
subscription = "<YOUR_SUBSCRIPTION_ID>"
resource_group = "<YOUR_RESOURCE_GROUP_NAME>"
resource_name = "<YOUR_AZURE_OPENAI_RESOURCE_NAME>"
model_deployment_name = "gpt-4o-ft"

deploy_params = {'api-version': "2023-05-01"}
deploy_headers = {'Authorization': 'Bearer {}'.format(token), 'Content-Type': 'application/json'}

deploy_data = {
    "sku": {"name": "standard", "capacity": 1},
    "properties": {
        "model": {
            "format": "OpenAI",
            "name": "<FINE_TUNED_MODEL>",
            "version": "1"
        }
    }
}
deploy_data = json.dumps(deploy_data)

request_url = f'https://management.azure.com/subscriptions/{subscription}/resourceGroups/{resource_group}/providers/Microsoft.CognitiveServices/accounts/{resource_name}/deployments/{model_deployment_name}'

print('Creating a new deployment...')

r = requests.put(request_url, params=deploy_params, headers=deploy_headers, data=deploy_data)

print(r)
print(r.reason)
print(r.json())

##### run the following code to use your customized model in a chat completion call

In [0]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
  api_key = os.getenv("AZURE_OPENAI_API_KEY"),
  api_version = "2024-02-01"
)

response = client.chat.completions.create(
    model = "gpt-4o-ft",
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Does Azure OpenAI support customer managed keys?"},
        {"role": "assistant", "content": "Yes, customer managed keys are supported by Azure OpenAI."},
        {"role": "user", "content": "Do other Azure AI services support this too?"}
    ]
)

print(response.choices[0].message.content)